In [29]:
from PIL import Image
import rasterio
import os
import numpy as np
import xml.etree.ElementTree as ET
import copy
import random
from shutil import copyfile, copy2, move
import shutil
from rasterio.warp import reproject, calculate_default_transform, Resampling
from rasterio.crs import CRS

In [2]:
def normalize(numpy_array, factor):
    mid_range = (np.percentile(numpy_array, 97))
    numpy_array = numpy_array / mid_range * 255
    numpy_array = np.clip(numpy_array, a_min = 0, a_max = 255)
    return numpy_array

In [38]:
def TifftoJPEGreproj(tiff):
    with rasterio.open(tiff) as raw:
        arr = raw.read()
        #arr = arr[[4,2,1],,]
        x = arr.copy()
        
        dest_CRS = CRS(init='epsg:3857')
        dst_affine, dst_width, dst_height = calculate_default_transform(
        raw.crs, dest_CRS, raw.width, raw.height, *raw.bounds)
        x = np.empty([arr.shape[0],dst_height, dst_width])
        
        reproject(source=arr,
              src_crs = raw.crs, 
              src_transform = raw.transform,
              destination = x,
              dst_transform = dst_affine,
              dst_crs = dest_CRS,
              resampling=Resampling.nearest,
              num_threads=2)
        
        r, g, b = x[4], x[2], x[1]
        
        r = normalize(r, 1.6)
        g = normalize(g, 1.6)
        b = normalize(b, 1.6)

        outData = np.stack((r,g,b), axis = 0)
        outData = outData.astype(np.uint8)
        
        name = os.path.split(tiff)[1].split('.')[0]
        
        outFile = os.path.join(pth, '%s.jpg' % name)
        
        jpeg = rasterio.open(outFile, 'w', driver="JPEG", dtype='uint8',
                                count=outData.shape[0],height=outData.shape[1],width=outData.shape[2])
        jpeg.write(outData)
        jpeg.close()
  

In [3]:
def TifftoJPEG(tiff):
    with rasterio.open(tiff) as raw:
        arr = raw.read()
        r, g, b = arr[4], arr[2], arr[1]
        r = normalize(r, 1.6)
        g = normalize(g, 1.6)
        b = normalize(b, 1.6)
        outData = np.stack((r,g,b), axis = 0)
        outData = outData.astype(np.uint8)
        name = os.path.split(tiff)[1].split('.')[0]
        outFile = os.path.join(pth, '%s.jpg' % name)
        jpeg = rasterio.open(outFile, 'w', driver="JPEG", dtype='uint8',
                                count=outData.shape[0],height=outData.shape[1],width=outData.shape[2])
        jpeg.write(outData)
        jpeg.close()

In [41]:
pth = r'D:\WWTP\GBDX_images\MechaTurk'

for root, dirs, files in os.walk(pth, topdown=False):
    for name in files:
        fil = os.path.join(root, name)
        if os.path.split(fil)[1].split('.')[-1] == 'tif':
            try:
                TifftoJPEGreproj(fil)
            except:
                pass

C:\Users\charl\AppData\Local\Continuum\anaconda3\envs\GGCW\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: The value of this property will change in version 1.0. Please see https://github.com/mapbox/rasterio/issues/86 for details.
  


In [4]:
def histeq(im,nbr_bins=256):
  """  Histogram equalization of a 1 band image. """

  # get image histogram
  imhist,bins = np.histogram(im.flatten(),nbr_bins,normed=True)
  cdf = imhist.cumsum() # cumulative distribution function
  cdf = 255 * cdf / cdf[-1] # normalize

  # use linear interpolation of cdf to find new pixel values
  im2 = np.interp(im.flatten(),bins[:-1],cdf)

  return im2.reshape(im.shape)

### Image Flipping

In [8]:
class BBOX:
    def __init__(self, element):
        self.name = element[0].text
        for e in element:
            if e.tag == 'bndbox':
                self.xmin = e[0].text
                self.ymin = e[1].text
                self.xmax = e[2].text
                self.ymax = e[3].text
    
    def shape(self):
        return 'Class: %s, Bounding box: (%s, %s, %s, %s)' % (self.name, self.xmin, self.ymin, self.xmax, self.ymax)
    
class DIMS:
    def __init__(self, element):
        for e in element:
            if e.tag == 'width':
                self.width = e.text
            elif e.tag == 'height':
                self.height = e.text
            elif e.tag == 'depth':
                self.depth = e.text
        self.dims = ('Shape: %s x %s, depth: %s' % (self.width, self.height, self.depth))
        
class IMAGE:
    
    def __init__(self, fname, path):
        self.fname = fname
        self.path = path
        self.outpath = os.path.join(self.path, 'Output')
        if os.path.exists(self.outpath) == False:
            os.mkdir(os.path.join(self.path, r'Output'))
        self.xml = os.path.join(path,self.fname.replace('.jpg','.xml'))
        self.outxml = os.path.join(self.outpath, self.fname.replace('.jpg','.xml'))
        self.xmltree = ET.parse(self.xml)
        self.root = self.xmltree.getroot()
        self.image = Image.open(os.path.join(self.path,self.fname))
        self.image.save(os.path.join(self.outpath, fname))
        self.xmltree.write(self.outxml)
        
        self.bbox_list, self.dims = [], []
        bboxes = self.root.findall('object')
        self.dims = DIMS(self.root.find('size'))
        for b in bboxes:
            self.bbox_list.append(BBOX(b))
        
    def Rotate90(self, bbox, dim, count):
        h1,k1 = int(bbox.xmin), int(bbox.ymin)
        h2,k2 = int(bbox.xmax), int(bbox.ymax)
        if count == 1:
            o = int(dim.width)
        else:
            o = int(dim.height)
        bbox.xmin, bbox.ymin = k1, (o - h1)
        bbox.xmax, bbox.ymax = k2, (o - h2)
        return bbox

    def CreateRotatedImages(self, degrees):
        
        newtree = copy.deepcopy(self.xmltree)
        root = newtree.getroot()
        
        ### Adjust JPEG
        rim = self.image.rotate(degrees, expand = True)
        newname = self.fname.replace('.jpg','_%s.jpg' % degrees)
        rim.save(os.path.join(self.outpath, newname))
        
        ### Adjust XML
        
        # Adjust bounding Box
        for obj in root.iter('object'):
            i = BBOX(obj)
            
            if degrees == 90:
                newbbox = self.Rotate90(i, self.dims, 1)
            elif degrees == 180:
                newbbox = self.Rotate90(i, self.dims, 1)
                newbbox = self.Rotate90(newbbox, self.dims, 2)
            elif degrees == 270:
                newbbox = self.Rotate90(i, self.dims, 1)
                newbbox = self.Rotate90(newbbox, self.dims, 2)
                newbbox = self.Rotate90(newbbox, self.dims, 1)
            for t in obj:
                if t.tag == 'bndbox':
                    t[0].text = str(min(newbbox.xmin, newbbox.xmax))
                    t[1].text = str(min(newbbox.ymin, newbbox.ymax))
                    t[2].text = str(max(newbbox.xmax, newbbox.xmin))
                    t[3].text = str(max(newbbox.ymax, newbbox.ymin))
        
        # Adjust dimensions
        if degrees == 90  or degrees == 270:
            for obj in root.iter("size"):
                w = obj.find("width")
                w.text = str(self.dims.height)
                h = obj.find("height")
                h.text = str(self.dims.width)
                
        # Adjust filename 
        for obj in root.iter("filename"):
            obj.text = str(newname)
        for obj in root.iter("path"):
            obj.text = str(os.path.join(self.path, newname))
        
        # Write new file
        newtree.write(self.outxml.replace(".xml", "_%s.xml" % str(degrees)))

    def Transpose(self, bbox, dim, flip):
        h1,k1 = int(bbox.xmin), int(bbox.ymin)
        h2,k2 = int(bbox.xmax), int(bbox.ymax)
        if flip == 'lr':
            o = int(self.dims.width)
            bbox.xmin, bbox.ymin = (o - h1), k1
            bbox.xmax, bbox.ymax = (o - h2), k2
        elif flip == 'tb':
            o = int(self.dims.height)
            bbox.xmin, bbox.ymin = h1, (o - k1)
            bbox.xmax, bbox.ymax = h2, (o - k2)
        return bbox
        
    def CreateFlippedImages(self, flip):
        
        newtree = copy.deepcopy(self.xmltree)
        root = newtree.getroot()
        
        ### Adjust JPEG
        if flip == 'lr':
            flim = self.image.transpose(method = Image.FLIP_LEFT_RIGHT)
        elif flip == 'tb':
            flim = self.image.transpose(method = Image.FLIP_TOP_BOTTOM)
        newname = self.fname.replace('.jpg','_%s.jpg' % flip)
        flim.save(os.path.join(self.outpath, newname))
        
        ### Adjust XML
        
        # Adjust bounding Box
        for obj in root.iter('object'):
            i = BBOX(obj)
            if flip == 'lr':
                newbbox = self.Transpose(i, self.dims, flip)
            elif flip == 'tb':
                newbbox = self.Transpose(i, self.dims, flip)
            for t in obj:
                if t.tag == 'bndbox':
                    t[0].text = str(min(newbbox.xmin, newbbox.xmax))
                    t[1].text = str(min(newbbox.ymin, newbbox.ymax))
                    t[2].text = str(max(newbbox.xmax, newbbox.xmin))
                    t[3].text = str(max(newbbox.ymax, newbbox.ymin))
                
        # Adjust filename 
        for obj in root.iter("filename"):
            obj.text = str(newname)
        for obj in root.iter("path"):
            obj.text = str(os.path.join(self.outpath, newname))
        
        # Write new file
        newtree.write(self.outxml.replace(".xml", "_%s.xml" % str(flip)))

### Run Image creation tool

In [10]:
pth = r'D:\WWTP\GBDX_images\Mexico'
for root, dirs, files in os.walk(pth, topdown=False):
    for name in files:
        fil = os.path.join(root, name)
        if os.path.split(name)[1].split('.')[-1] == 'jpg' and os.path.exists(os.path.join(root, os.path.split(name)[1].split('.')[-2]+'.xml')) == True:
            image = IMAGE(name, pth)
            image.CreateRotatedImages(90)
            image.CreateRotatedImages(180)
            image.CreateRotatedImages(270)
            image.CreateFlippedImages('lr')
            image.CreateFlippedImages('tb')

In [2]:
PATH = r'D:\WWTP\GBDX_images\Mexico\Output'
outtrain = r'C:\tensorflow1\models\research\object_detection\images\train'
outtest = r'C:\tensorflow1\models\research\object_detection\images\test'

### Move files into Tensorflow Suite with random Test / Train split

In [3]:
for root, dirs, files in os.walk(PATH, topdown=False):
    for name in files:
        tif = name.replace('.jpg','.xml')
        if os.path.split(name)[1].split('.')[-1] == 'jpg':
            if os.path.exists(os.path.join(outtest, name)) == True or os.path.exists(os.path.join(outtrain, name)):
                pass
            else:

                if random.randint(1,100) > 75:
                    shutil.copyfile(os.path.join(root, name), os.path.join(outtest, name))
                    shutil.copyfile(os.path.join(root, tif), os.path.join(outtest, tif))
                else:
                    shutil.copyfile(os.path.join(root, name), os.path.join(outtrain, name))
                    shutil.copyfile(os.path.join(root, tif), os.path.join(outtrain, tif))